## LionForests Multi Label Experiments: AI4I Dataset

In [1]:
import sys
cpath = !pwd
sys.path.append('/usr/src/app/algorithms/')
sys.path.append('/usr/src/app/')

Load Libraties

In [2]:
from sklearn.metrics import f1_score, precision_score
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import time
import warnings
import random
import numpy as np
from algorithms.anchor.anchor_tabular import AnchorTabularExplainer
from datasets.dataset import Dataset
import pandas as pd
from utilities.dummy_utilizer import DummyUtilizer
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from algorithms.MARLENA.marlena.marlena.marlena import MARLENA
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.CHIRPS.structures import data_container
import algorithms.CHIRPS.routines as rt
import algorithms.CHIRPS.structures as strcts
from scipy import sparse

np.seterr(invalid='ignore')
warnings.filterwarnings("ignore")

Load Dataset

In [3]:
ai4i = Dataset()
X, y, feature_names, label_names = ai4i.load_ai4i()

Initialize LionForests

In [4]:
from lionforestsmulti import LionForests
lf = LionForests(None, False, None, feature_names, label_names)

In [5]:
parameters = [{
    'max_depth': [10],
    'max_features': ['sqrt'],
    'bootstrap': [False],
    'min_samples_leaf' : [1],
    'n_estimators': [500]
}]
lf.fit(X, y, parameters, True)

Best RF model for this dataset

In [6]:
lf.model

RandomForestClassifier(bootstrap=False, max_depth=10, max_features='sqrt',
                       n_estimators=500, n_jobs=-1, random_state=0)

Example explanation for option "all predicted labelsets"

In [7]:
lf.explain_n_wise(X[2],'all',True, method='1')[0]

'if 298.55<=Air temperature [K]<=298.95 & 308.85<=Process temperature [K]<=308.95 & 1448.0<=Rotational speed [rpm]<=1468.0 & 33.2<=Torque [Nm]<=41.45 & 207.5<=Tool wear [min]<=211.5 then TWF'

Example explanation for option "per label"

In [8]:
lf.explain_n_wise(X[2],'per label',True, method='1')['TWF'][0]

'if 298.55<=Air temperature [K]<=298.95 & 308.85<=Process temperature [K]<=308.95 & 1448.0<=Rotational speed [rpm]<=1468.0 & 33.2<=Torque [Nm]<=41.45 & 207.5<=Tool wear [min]<=211.5 then TWF'

Example explanation for option "frequent pairs"

In [9]:
lf.explain_n_wise(X[2],'frequent pairs',7, True, method='1')[('TWF',)][0]

'if 298.55<=Air temperature [K]<=298.95 & 308.85<=Process temperature [K]<=308.95 & 1448.0<=Rotational speed [rpm]<=1468.0 & 33.2<=Torque [Nm]<=41.45 & 207.5<=Tool wear [min]<=211.5 then TWF'

## Experiments

1. Comparison between LF variations

In [ ]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names, tech=False, random_state=10):
    parameters = [{
        'max_depth': [10],
        'max_features': ['sqrt'],
        'bootstrap': [False],
        'min_samples_leaf': [1],
        'n_estimators': [500]
    }]
    lf = LionForests(None, False, None, feature_names, class_names)
    lf.fit(X_train, y_train, params=parameters)

    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)

    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task, random_state=10):

        # LionForests
        def lf_rule_all(instance):
            temp = lf.explain_n_wise(instance, 'all')[5]
            rule = {}
            for key, value in temp.items():
                rule[key] = [['<=', value[1]], ['>', value[0]]]
            return rule

        def lf_rule_per_label(instance):
            temp = lf.explain_n_wise(instance, 'per label')
            rules = {}
            for key_o in list(temp.keys()):
                rule = {}
                for key, value in temp[key_o][5].items():
                    rule[key] = [['<=', value[1]], ['>', value[0]]]
                rules[key_o] = rule
            return rules

        def lf_rule_pairs(instance):
            temp = lf.explain_n_wise(
                instance, 'frequent pairs', len(class_names))
            rules = {}
            for key_o in list(temp.keys()):
                rule = {}
                for key, value in temp[key_o][5].items():
                    rule[key] = [['<=', value[1]], ['>', value[0]]]
                rules[key_o] = rule
            return rules

        return {'lf-a': lf_rule_all, 'lf-l': lf_rule_per_label, 'lf-p': lf_rule_pairs}

    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf,
                                'classification', random_state)
    if tech:
        return interpretation, lf

    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] <= rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] < rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    rule_generator = interpretation
    full_coverage = {'lf-a': 0, 'lf-l': 0, 'lf-p': 0}
    rule_length = {'lf-a': 0, 'lf-l': 0, 'lf-p': 0}
    f_precision = {'lf-a': [], 'lf-l': [], 'lf-p': []}
    time_response = {'lf-a': [], 'lf-l': [], 'lf-p': []}
    rules = {'lf-a': [], 'lf-l': [], 'lf-p': []}

    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions

    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)
    skippedkataskeywastwnplunthriousunistounskipmadame = 0

    for tesd_ind in range(len(test)):
        # or trees in lf.model.estimators_:
        #   print(trees.predict_proba([x_test_temp_lf[tesd_ind]]))
        # rint(y_test_temp[tesd_ind])
        if np.sum(y_test_temp[tesd_ind]) == 0:
            #print(tesd_ind,'Kataskeuastes plunthriwn sunistoun skip')
            skippedkataskeywastwnplunthriousunistounskipmadame += 1
        else:
            #print(tesd_ind,'sunexise madam')
            for name, method in rule_generator.items():
                if name == 'lf-a':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    coverage = 0
                    precision = []
                    co = 0
                    for i in x_test_temp_lf:
                        res = rule_cov_LF(i, feature_names, rule)
                        coverage = coverage + res
                        if res == 1:
                            precision.append(
                                [list(y_test_temp[tesd_ind]), list(y_test_temp[co])])
                        co = co + 1
                    if len(precision) >= 1:
                        precision = np.array(precision)
                        f_precision[name].append(precision_score(
                            precision[:, 0], precision[:, 1], average='micro'))
                    full_coverage[name] = full_coverage[name] + \
                        coverage/len(x_test_temp_lf)
                    rules[name].append(rule)
                    len_rule = len(rule)
                elif name == 'lf-l':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        label_index = list(label_names).index(key)
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp_lf:
                            res = rule_cov_LF(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                precision.append(
                                    [y_test_temp[tesd_ind][label_index], y_test_temp[co][label_index]])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, :1], precision[:, 1:], average='micro'))
                        total_coverage += coverage/len(x_test_temp_lf)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)
                elif name == 'lf-p':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        labeles = []
                        for kk in key:
                            labeles.append(list(label_names).index(kk))
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp_lf:
                            res = rule_cov_LF(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                temp_prediction = [y_test_temp[tesd_ind][j] for j in range(
                                    len(y_test_temp[tesd_ind])) if j in labeles]
                                temperatura = [y_test_temp[co][j] for j in range(
                                    len(y_test_temp[co])) if j in labeles]
                                precision.append(
                                    [temp_prediction, temperatura])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, 0], precision[:, 1], average='micro'))
                        total_coverage += coverage/len(x_test_temp_lf)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)

                time_response[name].append(te)
                rule_length[name] = rule_length[name] + len_rule
    return rule_generator, full_coverage, rule_length, f_precision, time_response, skippedkataskeywastwnplunthriousunistounskipmadame

In [ ]:
total_results = []
kf = KFold(n_splits=10, random_state = 77)
folds = 0
test_size = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]
    print('# of Fold: ' + str(folds+1) + ', size of test: ' + str(len(X_test)))    
    results = measure(X_train, X_test, y_train, y_test, feature_names, label_names)
    test_size.append(len(X_test)-results[-1])
    total_results.append(results)
    folds=folds+1

Results

In [ ]:
full_coverage = {'lf-a':[], 'lf-l':[], 'lf-p':[]}
rule_length = {'lf-a':[], 'lf-l':[], 'lf-p':[]}
f_precision ={'lf-a':[], 'lf-l':[], 'lf-p':[]}
f_time = {'lf-a':[], 'lf-l':[], 'lf-p':[]}
k = 0
for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/test_size[k])
        rule_length[name].append(i[2][name]/test_size[k])
        l = [k for k in i[3][name] if str(k) != str(np.average([])) ]
        f_precision[name].append(np.array(l).mean())
        f_time[name].append(np.array(i[4][name]).mean())
    k = + 1
for name, method in total_results[0][0].items():
    print(name,  '| %5.4f  %5.3f | %5.4f %5.3f | %5.4f  %5.3f | %5.4f  %5.3f' 
          % (np.array(full_coverage[name]).mean(),np.array(full_coverage[name]).std(),
             np.array(rule_length[name]).mean(),np.array(rule_length[name]).std(),
             np.array(f_precision[name]).mean(),np.array(f_precision[name]).std(),
             np.array(f_time[name]).mean(),np.array(f_time[name]).std()))

2. Comparison between LF-a and multi-label explainability techniques

In [ ]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names, tech=False, random_state=10):
    parameters = [{
        'max_depth': [10],
        'max_features': ['sqrt'],
        'bootstrap': [False],
        'min_samples_leaf': [1],
        'n_estimators': [500]
    }]
    lf = LionForests(None, False, None, feature_names, class_names)
    lf.fit(X_train, y_train, params=parameters)

    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)

    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task, random_state=10):

        # LionForests
        def lf_rule_all(instance):
            temp = lf.explain_n_wise(instance, 'all')[5]
            rule = {}
            for key, value in temp.items():
                rule[key] = [['<=', value[1]], ['>', value[0]]]
            return rule

        gt = GlobalSurrogateTree(
            train, predictions, feature_names, task, random_state)
        print('    GT Ready')
        lt = LocalSurrogateTree(
            train, predictions, feature_names, task, 150, random_state)

        def marlena(instance):
            m1 = MARLENA(neigh_type='mixed', random_state=42)
            i2e = pd.Series(instance, index=feature_names)
            X2E = pd.DataFrame(train, columns=feature_names)
            rule, _, _, _, _ = m1.extract_explanation(i2e, X2E, lf.model, feature_names, [],
                                                      class_names, k=10, size=50, alpha=0.7)

            path = rule.split('->')[0][1:-2]
            prediction = rule.split('->')[1][1:]
            conjuctions = {}
            if '{}' in rule:
                print(rule)
            else:
                for conj in path.split('\n'):
                    temp_conj = conj.strip(',')
                    if temp_conj[0] == ' ':
                        temp_conj = temp_conj[1:]
                    if '>' in temp_conj:
                        temp = temp_conj.split(' > ')
                        if temp[0] in conjuctions:
                            position = 0
                            if len(conjuctions[temp[0]]) == 2:
                                if conjuctions[temp[0]][1][0] == '>':
                                    position = 1
                            if conjuctions[temp[0]][position][0] == '>':
                                if conjuctions[temp[0]][position][1] > float(temp[1]):
                                    conjuctions[temp[0]][position][1] = float(
                                        temp[1])
                            elif conjuctions[temp[0]][0][0] == '<=':
                                conjuctions[temp[0]].append(
                                    ['>', float(temp[1])])

                        else:
                            conjuctions[temp[0]] = [['>', float(temp[1])]]
                    else:
                        temp = temp_conj.split(' <= ')
                        if temp[0] in conjuctions:
                            position = 0
                            if len(conjuctions[temp[0]]) == 2:
                                if conjuctions[temp[0]][1][0] == '<=':
                                    position = 1

                            if conjuctions[temp[0]][position][0] == '<=':
                                if conjuctions[temp[0]][position][1] <= float(temp[1]):
                                    conjuctions[temp[0]][position][1] = float(
                                        temp[1])
                            elif conjuctions[temp[0]][0][0] == '>':
                                conjuctions[temp[0]].append(
                                    ['<=', float(temp[1])])
                        else:
                            conjuctions[temp[0]] = [['<=', float(temp[1])]]
            local_prediction = np.zeros((len(label_names)), dtype=int)
            for label in range(len(label_names)):
                if label_names[label] in prediction:
                    local_prediction[label] = 1
            return (conjuctions, local_prediction)

        return {'lf-a': lf_rule_all, 'gs': gt.rule, 'ls': lt.rule, 'ma': marlena}

    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf,
                                'classification', random_state)
    if tech:
        return interpretation, lf

    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] <= rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] < rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    rule_generator = interpretation
    full_coverage = {'lf-a': 0, 'ls': 0, 'gs': 0, 'ma': 0}
    rule_length = {'lf-a': 0, 'ls': 0, 'gs': 0, 'ma': 0}
    f_precision = {'lf-a': [], 'ls': [], 'gs': [], 'ma': []}
    time_response = {'lf-a': [], 'ls': [], 'gs': [], 'ma': []}
    rules = {'lf-a': [], 'ls': [], 'gs': [], 'ma': []}

    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions

    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)
    skippedkataskeywastwnplunthriousunistounskipmadame = 0

    for tesd_ind in range(len(test)):
        # or trees in lf.model.estimators_:
        #   print(trees.predict_proba([x_test_temp_lf[tesd_ind]]))
        # rint(y_test_temp[tesd_ind])
        if np.sum(y_test_temp[tesd_ind]) == 0:
            #print(tesd_ind,'Kataskeuastes plunthriwn sunistoun skip')
            skippedkataskeywastwnplunthriousunistounskipmadame += 1
        else:
            #print(tesd_ind,'sunexise madam')
            for name, method in rule_generator.items():
                if name == 'lf-a':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    coverage = 0
                    precision = []
                    co = 0
                    for i in x_test_temp_lf:
                        res = rule_cov_LF(i, feature_names, rule)
                        coverage = coverage + res
                        if res == 1:
                            precision.append(
                                [list(y_test_temp[tesd_ind]), list(y_test_temp[co])])
                        co = co + 1
                    if len(precision) >= 1:
                        precision = np.array(precision)
                        f_precision[name].append(precision_score(
                            precision[:, 0], precision[:, 1], average='micro'))
                    full_coverage[name] = full_coverage[name] + \
                        coverage/len(x_test_temp_lf)
                    rules[name].append(rule)
                    len_rule = len(rule)
                elif name == 'ls' or name == 'gs' or name == 'ma':
                    ts = time.time()
                    rule, prediction = method(x_test_temp[tesd_ind])
                    te = time.time() - ts
                    coverage = 0
                    precision = []
                    co = 0
                    for i in x_test_temp:
                        res = rule_cov(i, feature_names, rule)
                        coverage = coverage + res
                        if res == 1:
                            precision.append(
                                [list(prediction), list(y_test_temp[co])])
                        co = co + 1
                    if len(precision) >= 1:
                        precision = np.array(precision)
                        f_precision[name].append(precision_score(
                            precision[:, 0], precision[:, 1], average='micro'))
                    full_coverage[name] = full_coverage[name] + \
                        coverage/len(x_test_temp)
                    rules[name].append(rule)
                    len_rule = len(rule)
                time_response[name].append(te)
                rule_length[name] = rule_length[name] + len_rule
    return rule_generator, full_coverage, rule_length, f_precision, time_response, skippedkataskeywastwnplunthriousunistounskipmadame

In [ ]:
total_results = []
kf = KFold(n_splits=10, random_state = 77)
folds = 0
test_size = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]
    print('# of Fold: ' + str(folds+1) + ', size of test: ' + str(len(X_test)))    
    results = measure(X_train, X_test, y_train, y_test, feature_names, label_names)
    test_size.append(len(X_test)-results[-1])
    total_results.append(results)
    folds=folds+1

Results

In [ ]:
full_coverage = {'lf-a':[], 'ls':[], 'gs':[], 'ma':[]}
rule_length = {'lf-a':[], 'ls':[], 'gs':[], 'ma':[]}
f_precision ={'lf-a':[], 'ls':[], 'gs':[], 'ma':[]}
f_time = {'lf-a':[], 'ls':[], 'gs':[], 'ma':[]}
k = 0
for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/test_size[k])
        rule_length[name].append(i[2][name]/test_size[k])
        l = [k for k in i[3][name] if str(k) != str(np.average([])) ]
        f_precision[name].append(np.array(l).mean())
        f_time[name].append(np.array(i[4][name]).mean())
    k = + 1
for name, method in total_results[0][0].items():
    print(name,  '| %5.4f  %5.3f | %5.4f %5.3f | %5.4f  %5.3f | %5.4f  %5.3f' 
          % (np.array(full_coverage[name]).mean(),np.array(full_coverage[name]).std(),
             np.array(rule_length[name]).mean(),np.array(rule_length[name]).std(),
             np.array(f_precision[name]).mean(),np.array(f_precision[name]).std(),
             np.array(f_time[name]).mean(),np.array(f_time[name]).std()))

3. Comparison between LF-l and "per label" explainability techniques (CHIRPS/ANCHORS)

In [ ]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names, tech=False, random_state=10):
    parameters = [{
        'max_depth': [10],
        'max_features': ['sqrt'],
        'bootstrap': [False],
        'min_samples_leaf': [1],
        'n_estimators': [500]
    }]
    scaler = MinMaxScaler(feature_range=(-1, 1))
    lf = LionForests(None, False, scaler, feature_names, class_names)
    lf.fit(X_train, y_train, params=parameters)

    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)

    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task, random_state=10):

        def lf_rule_per_label(instance):
            temp = lf.explain_n_wise(instance, 'per label')
            rules = {}
            for key_o in list(temp.keys()):
                rule = {}
                for key, value in temp[key_o][5].items():
                    rule[key] = [['<=', value[1]], ['>', value[0]]]
                rules[key_o] = rule
            return rules

        inddd = 0
        anchors = {}
        for label_name in label_names:
            explainer = AnchorTabularExplainer(
                ['not '+label_name, label_name], feature_names, train)
            anchors[inddd] = explainer
            inddd += 1

        def anchors_method(instance):
            prediction = lf.model.predict([instance])[0]
            rules = {}
            for pred in range(len(prediction)):
                if prediction[pred] == 1:
                    explainer = anchors[pred]

                    def my_model(x):
                        return lf.model.predict(x)[:, pred]
                    exp = explainer.explain_instance(
                        instance, my_model, threshold=0.95)
                    anchors_dict = {}
                    for i in exp.names():
                        terms = i.split(' ')
                        if len(terms) == 3:
                            anchors_dict[terms[0]] = [
                                [terms[1], float(terms[2])]]
                        else:
                            anchors_dict[terms[2]] = [[terms[3], float(terms[4])], [
                                terms[1], float(terms[0])]]
                    rules[class_names[pred]] = anchors_dict
            return rules

        chts = time.time()
        chirps = {}
        inddd = 0
        for label_name in label_names:
            explainer = AnchorTabularExplainer(
                ['not '+label_name, label_name], feature_names, train)
            # CHIRPS =======================================================================================
            project_dir = 'C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\CHIRPS'
            temp_frame = pd.DataFrame(np.hstack((train, y_train[:, inddd].reshape(
                len(y_train), 1))), columns=feature_names+['class'])
            temp_frame['class'] = temp_frame['class'].astype(int)
            #temp_frame = temp_frame.replace({"class": {1: 2}})
            #temp_frame = temp_frame.replace({"class": {0: 1}})

            mydata = data_container(
                data=temp_frame, class_col='class', var_names=feature_names,
                project_dir=project_dir, save_dir='flags'+str(inddd), random_state=random_state)
            meta_data = mydata.get_meta()
            f_walker = strcts.classification_trees_walker(
                forest=model, inddd=inddd, meta_data=meta_data)
            f_walker.forest_walk(instances=test, labels=model.predict(test)[
                                 :, inddd], forest_walk_async=False)

            explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                                   forest=model,
                                                   # any representative sample can be used
                                                   sample_instances=sparse.csr_matrix(
                                                       train),
                                                   sample_labels=predictions[:, inddd],
                                                   meta_data=meta_data)
            explanations.run_explanations(target_classes=model.predict(test)[:, inddd],  # we're explaining the prediction, not the true label!
                                          explanation_async=False,
                                          random_state=random_state,
                                          which_trees='majority',
                                          alpha_paths=0.0,
                                          support_paths=0.1,
                                          score_func=1,
                                          precis_threshold=0.9,
                                          disc_path_bins=4,
                                          merging_bootstraps=20,
                                          pruning_bootstraps=20,
                                          delta=0.2,
                                          weighting='kldiv')
            chirps[inddd] = explanations
            inddd += 1
        chte = (time.time() - chts)/len(test)

        def chirps_method(instance, idx):
            prediction = lf.model.predict([instance])[0]
            rules = {}
            for pred in range(len(prediction)):
                if prediction[pred] == 1:
                    chirps_dict = {}
                    explanations = chirps[pred]
                    for i in explanations.explainers[idx].pruned_rule:
                        if i[1]:
                            chirps_dict[i[0]] = [['<=', float(i[2])]]
                        else:
                            chirps_dict[i[0]] = [['>', float(i[2])]]
                    rules[class_names[pred]] = chirps_dict
            return rules, 0, chte

        return {'lf-l': lf_rule_per_label, 'an': anchors_method, 'ch': chirps_method}

    interpretation = techniques(lf.model, train, y_train, predictions,
                                test, feature_names, class_names, lf, 'classification', random_state)
    if tech:
        return interpretation, lf

    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] <= rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
                    if instance[k] < rule[feature][1][1]:  # THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]:  # <=
                        return 0
        return 1

    rule_generator = interpretation
    full_coverage = {'lf-l': 0, 'an': 0, 'ch': 0}
    rule_length = {'lf-l': 0, 'an': 0, 'ch': 0}
    f_precision = {'lf-l': [], 'an': [], 'ch': []}
    time_response = {'lf-l': [], 'an': [], 'ch': []}
    rules = {'lf-l': [], 'an': [], 'ch': []}

    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions

    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)

    skippedkataskeywastwnplunthriousunistounskipmadame = 0

    for tesd_ind in range(len(test)):
        if np.sum(y_test_temp[tesd_ind]) == 0:
            skippedkataskeywastwnplunthriousunistounskipmadame += 1
        else:
            for name, method in rule_generator.items():
                if name == 'lf-l':
                    ts = time.time()
                    rule = method(x_test_temp_lf[tesd_ind])
                    te = time.time() - ts
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        label_index = list(label_names).index(key)
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp_lf:
                            res = rule_cov_LF(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                precision.append(
                                    [y_test_temp[tesd_ind][label_index], y_test_temp[co][label_index]])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, :1], precision[:, 1:], average='micro'))
                        total_coverage += coverage/len(x_test_temp_lf)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)
                elif name == 'an':
                    ts = time.time()
                    rule = method(x_test_temp[tesd_ind])
                    te = time.time() - ts
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        label_index = list(label_names).index(key)
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp:
                            res = rule_cov(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                precision.append(
                                    [y_test_temp[tesd_ind][label_index], y_test_temp[co][label_index]])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, :1], precision[:, 1:], average='micro'))
                        total_coverage += coverage/len(x_test_temp)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)
                elif name == 'ch':
                    rule, op, te = method(x_test_temp[tesd_ind], tesd_ind)
                    total_coverage = 0
                    total_precision = []
                    len_rule = 0
                    for key in rule.keys():
                        temp_rule = rule[key]
                        len_rule += len(temp_rule)
                        label_index = list(label_names).index(key)
                        coverage = 0
                        precision = []
                        co = 0
                        for i in x_test_temp:
                            res = rule_cov(i, feature_names, temp_rule)
                            coverage = coverage + res
                            if res == 1:
                                precision.append(
                                    [y_test_temp[tesd_ind][label_index], y_test_temp[co][label_index]])
                            co = co + 1
                        if len(precision) >= 1:
                            precision = np.array(precision)
                            total_precision.append(precision_score(
                                precision[:, :1], precision[:, 1:], average='micro'))
                        total_coverage += coverage/len(x_test_temp)
                    total_precision = [k for k in total_precision if str(
                        k) != str(np.average([]))]
                    f_precision[name].append(np.array(total_precision).mean())
                    full_coverage[name] = full_coverage[name] + \
                        total_coverage/len(rule.keys())
                    rules[name].append(rule)

                time_response[name].append(te)
                rule_length[name] = rule_length[name] + len_rule
    return rule_generator, full_coverage, rule_length, f_precision, time_response, skippedkataskeywastwnplunthriousunistounskipmadame

In [ ]:
total_results = []
kf = KFold(n_splits=10, random_state = 77)
folds = 0
test_size = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]
    print('# of Fold: ' + str(folds+1) + ', size of test: ' + str(len(X_test)))    
    results = measure(X_train, X_test, y_train, y_test, [i.replace(' ','_') for i in feature_names], label_names)
    test_size.append(len(X_test)-results[-1])
    total_results.append(results)
    folds=folds+1

Results

In [ ]:
full_coverage = {'lf-l':[], 'an':[], 'ch':[]}
rule_length = {'lf-l':[], 'an':[], 'ch':[]}
f_precision = {'lf-l':[], 'an':[], 'ch':[]}
f_time = {'lf-l':[], 'an':[], 'ch':[]}
k = 0
for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/test_size[k])
        rule_length[name].append(i[2][name]/test_size[k])
        l = [k for k in i[3][name] if str(k) != str(np.average([])) ]
        f_precision[name].append(np.array(l).mean())
        f_time[name].append(np.array(i[4][name]).mean())
    k = + 1
for name, method in total_results[0][0].items():
    print(name,  '| %5.4f  %5.3f | %5.4f %5.3f | %5.4f  %5.3f | %5.4f  %5.3f' 
          % (np.array(full_coverage[name]).mean(),np.array(full_coverage[name]).std(),
             np.array(rule_length[name]).mean(),np.array(rule_length[name]).std(),
             np.array(f_precision[name]).mean(),np.array(f_precision[name]).std(),
             np.array(f_time[name]).mean(),np.array(f_time[name]).std()))

## Qualitative Experiments

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=5)

In [14]:
from utilities.dummy_utilizer import DummyUtilizer
lf = LionForests(None, False, DummyUtilizer(), feature_names, label_names)#,'x2'])

In [15]:
parameters = [{
    'max_depth': [5],
    'max_features': [0.75],
    'bootstrap': [False],
    'min_samples_leaf' : [1],
    'n_estimators': [500]
}]
lf.fit(x_train, y_train, parameters, True)

For a specific instance

In [16]:
idx = 84
instance = x_train[idx]

Let's print the prediction, and the actual labels

In [18]:
lf.model.predict(lf.utilizer.transform(x_train))[idx], y_train[idx]

(array([1, 0, 1, 1]), [1, 0, 1, 1])

We now print the LF rules

In [19]:
lf.explain_n_wise(instance,'all')

['if 2.5<=Type<=3.0 & 300.6<=Air temperature [K]<=301.65 & 310.05<=Process temperature [K]<=313.7 & 1351.0<=Rotational speed [rpm]<=1380.0 & 65.2<=Torque [Nm]<=65.5 & 207.5<=Tool wear [min]<=209.0 then TWF PWF OSF',
 500,
 6,
 251,
 6,
 {'Type': [2.5, 3.0],
  'Air temperature [K]': [300.6000061035156, 301.65000915527344],
  'Process temperature [K]': [310.0500030517578, 313.7],
  'Rotational speed [rpm]': [1351.0, 1380.0],
  'Torque [Nm]': [65.20000076293945, 65.5],
  'Tool wear [min]': [207.5, 209.0]}]

In [20]:
lf.explain_n_wise(instance,'per label')

{'TWF': ['if 2.5<=Type<=3.0 & 295.6<=Air temperature [K]<=301.65 & 1322.5<=Rotational speed [rpm]<=1419.5 & 65.2<=Torque [Nm]<=76.2 & 206.5<=Tool wear [min]<=251.0 then TWF',
  500,
  6,
  251,
  5,
  {'Type': [2.5, 3.0],
   'Air temperature [K]': [295.6, 301.65000915527344],
   'Rotational speed [rpm]': [1322.5, 1419.5],
   'Torque [Nm]': [65.20000076293945, 76.2],
   'Tool wear [min]': [206.5, 251.0]}],
 'PWF': ['if 2.5<=Type<=3.0 & 295.6<=Air temperature [K]<=301.65 & 1351.0<=Rotational speed [rpm]<=1380.0 & 65.2<=Torque [Nm]<=76.2 & 188.0<=Tool wear [min]<=251.0 then PWF',
  500,
  6,
  270,
  5,
  {'Type': [2.5, 3.0],
   'Air temperature [K]': [295.6, 301.65000915527344],
   'Rotational speed [rpm]': [1351.0, 1380.0],
   'Torque [Nm]': [65.20000076293945, 76.2],
   'Tool wear [min]': [188.0, 251.0]}],
 'OSF': ['if 2.5<=Type<=3.0 & 300.6<=Air temperature [K]<=300.75 & 65.2<=Torque [Nm]<=65.5 & 207.5<=Tool wear [min]<=251.0 then OSF',
  500,
  6,
  251,
  4,
  {'Type': [2.5, 3.0],
 

In [22]:
lf.explain_n_wise(instance,'frequent pairs', 10)[('PWF', 'OSF')]

['if 2.5<=Type<=3.0 & 295.6<=Air temperature [K]<=301.65 & 1351.0<=Rotational speed [rpm]<=1380.0 & 65.2<=Torque [Nm]<=76.2 & 185.0<=Tool wear [min]<=251.0 then PWF OSF',
 500,
 6,
 270,
 5,
 {'Type': [2.5, 3.0],
  'Air temperature [K]': [295.6, 301.65000915527344],
  'Rotational speed [rpm]': [1351.0, 1380.0],
  'Torque [Nm]': [65.20000076293945, 76.2],
  'Tool wear [min]': [185.0, 251.0]}]

And now the rules from the Global and Local Surrogate

In [24]:
gt = GlobalSurrogateTree(x_train, lf.model.predict(x_train), feature_names, 'classification')
lt = LocalSurrogateTree(x_train, lf.model.predict(x_train), feature_names, 'classification', 50)

In [513]:
gt.rule(instance)

({'Air temperature [K]': [['<=', 301.65000915527344]],
  'Tool wear [min]': [['>', 176.5]],
  'Torque [Nm]': [['>', 65.20000076293945]]},
 array([0, 0, 1, 1]))

In [28]:
lt.rule(instance)

({'Torque [Nm]': [['>', 48.400001525878906]],
  'Tool wear [min]': [['>', 188.0]],
  'Type': [['>', 2.5]]},
 array([1, 0, 1, 1]))

We are also using MARLENA

In [515]:
m1 = MARLENA(neigh_type='mixed',random_state=42)
i2e = pd.Series(instance,index=feature_names)
X2E = pd.DataFrame(x_train,columns=feature_names)
rule, _, _, _, _ = m1.extract_explanation(i2e ,X2E, lf.model, feature_names, [],
                                                      label_names, k=10, size=50, alpha=0.7)

In [516]:
rule

"{Air temperature [K] <= 303.0,\n Type > 2.97,\n Rotational speed [rpm] <= 1382.38} -> ['TWF' 'PWF' 'OSF']"

Now, we are extracting per label explanations from Anchors and CHIRPS

In [517]:
inddd = 0
anchors = {}
for label_name in label_names:
    explainer = AnchorTabularExplainer(['not '+label_name, label_name], feature_names, lf.utilizer.transform(X))
    anchors[inddd] = explainer
    inddd += 1        
def anchors_method(instance):
    prediction = lf.model.predict([instance])[0]
    rules = []
    for pred in range(len(prediction)):
        if prediction[pred]==1:
            explainer = anchors[pred]
            def my_model(x):
                return lf.model.predict(x)[:,pred]
            exp = explainer.explain_instance(instance, my_model, threshold=0.95)
            rules.append([label_names[pred],exp])
    return rules

In [519]:
anchor_explanations = anchors_method(instance)
anchor_explanations[0][0],anchor_explanations[0][1].exp_map['names']

('TWF',
 ['Tool wear [min] > 207.50',
  'Type > 2.00',
  'Air temperature [K] <= 301.60',
  'Torque [Nm] > 61.20',
  'Rotational speed [rpm] <= 1365.00'])

In [520]:
anchor_explanations[1][0],anchor_explanations[1][1].exp_map['names']

('PWF',
 ['Air temperature [K] <= 301.60',
  'Torque [Nm] > 61.20',
  'Type > 2.00',
  '309.50 < Process temperature [K] <= 311.20',
  '1326.50 < Rotational speed [rpm] <= 1365.00'])

In [521]:
anchor_explanations[2][0],anchor_explanations[2][1].exp_map['names']

('OSF', ['Tool wear [min] > 207.50', 'Torque [Nm] > 61.20'])

And CHIRPS

In [ ]:
chirps = {}
inddd = 0 
y_train = np.array(y_train)
x_train = np.float32(x_train)
for label_name in label_names:
    #CHIRPS =======================================================================================
    project_dir = 'C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\CHIRPS'
    temp_frame = pd.DataFrame(np.hstack((x_train, y_train[:,inddd].reshape(len(y_train),1))),
                              columns=feature_names+['class'])
    temp_frame['class']=temp_frame['class'].astype(int)

    mydata = data_container(
            data = temp_frame, class_col = 'class', var_names = feature_names,
            project_dir = project_dir, save_dir = 'flag'+str(inddd), random_state=123)
    meta_data = mydata.get_meta()
    f_walker = strcts.classification_trees_walker(forest=lf.model, inddd = inddd, meta_data=meta_data)
    f_walker.forest_walk(instances = x_train, labels = lf.model.predict(x_train)[:,inddd], forest_walk_async=False)

    explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                    forest=lf.model,
                                    sample_instances=sparse.csr_matrix(x_train), # any representative sample can be used
                                    sample_labels=lf.model.predict(x_train)[:,inddd],
                                    meta_data=meta_data)
    explanations.run_explanations(target_classes=lf.model.predict(x_train)[:,inddd], # we're explaining the prediction, not the true label!
                            explanation_async=False,
                            random_state=123,
                            which_trees='majority',
                            alpha_paths=0.0,
                            support_paths=0.1,
                            score_func=1,
                            precis_threshold=0.9,
                            disc_path_bins=4,
                            merging_bootstraps=20,
                            pruning_bootstraps=20,
                            delta=0.2,
                            weighting='kldiv')
    chirps[inddd] = explanations
    inddd += 1

In [49]:
def chirps_method(instance, idx):
    prediction = lf.model.predict([instance])[0]
    rules = {}
    for pred in range(len(prediction)):
        if prediction[pred]==1:
            chirps_dict = {}
            explanations = chirps[pred]
            for i in explanations.explainers[idx].pruned_rule:
                if i[1]:
                    chirps_dict[i[0]] = [['<=',float(i[2])]]
                else:
                    chirps_dict[i[0]] = [['>',float(i[2])]]
            rules[label_names[pred]] = chirps_dict
    return rules, 0

chirps_method(instance,idx)

({'TWF': {},
  'PWF': {'Air temperature [K]': [['<=', 302.55]],
   'Torque [Nm]': [['>', 65.03305]]},
  'OSF': {'Tool wear [min]': [['>', 176.53688]],
   'Torque [Nm]': [['>', 65.03456]]}},
 0)

Let's try to change a few values!

In [52]:
#Type:
for val in range(0, 3, 1):
    tc = instance.copy()
    tc[0]= val
    vv = lf.model.predict([tc, instance])
    print(val, vv[0], vv[1])

0 [0 0 0 1] [1 0 1 1]
1 [0 0 0 1] [1 0 1 1]
2 [0 0 0 1] [1 0 1 1]


In [50]:
#Air temperature [K]:
for val in list(range(2956, 3006, 1))+list(range(3013,3044,1)):
    tc = instance.copy()
    tc[1]= val/10
    vv = lf.model.predict([tc, instance])
    if 1 in (vv[0]-vv[1]) or -1 in (vv[0]-vv[1]):
        print(val/10, vv[0],vv[1])

301.7 [0 0 0 1] [1 0 1 1]
301.8 [0 0 0 1] [1 0 1 1]
301.9 [0 0 0 1] [1 0 1 1]
302.0 [0 0 0 1] [1 0 1 1]
302.1 [0 0 0 1] [1 0 1 1]
302.2 [0 0 0 1] [1 0 1 1]
302.3 [0 0 0 1] [1 0 1 1]
302.4 [0 0 0 1] [1 0 1 1]
302.5 [0 0 0 1] [1 0 1 1]
302.6 [0 0 0 1] [1 0 1 1]
302.7 [0 0 0 1] [1 0 1 1]
302.8 [0 0 0 1] [1 0 1 1]
302.9 [0 0 0 1] [1 0 1 1]
303.0 [0 0 0 1] [1 0 1 1]
303.1 [0 0 0 1] [1 0 1 1]
303.2 [0 0 0 1] [1 0 1 1]
303.3 [0 0 0 1] [1 0 1 1]
303.4 [0 0 0 1] [1 0 1 1]
303.5 [0 0 0 1] [1 0 1 1]
303.6 [0 0 0 1] [1 0 1 1]
303.7 [0 0 0 1] [1 0 1 1]
303.8 [0 0 0 1] [1 0 1 1]
303.9 [0 0 0 1] [1 0 1 1]
304.0 [0 0 0 1] [1 0 1 1]
304.1 [0 0 0 1] [1 0 1 1]
304.2 [0 0 0 1] [1 0 1 1]
304.3 [0 0 0 1] [1 0 1 1]


In [56]:
#Process temperature [K]:
for val in list(range(3061, 3100, 1)):#+list(range(3137,3137,1)):
    tc = instance.copy()
    tc[2]= val/10
    vv = lf.model.predict([tc, instance])
    if 1 in (vv[0]-vv[1]) or -1 in (vv[0]-vv[1]):
        print(val, vv[0],vv[1])

In [57]:
# Rotational speed [rpm]:
for val in list(range(1181, 1351)) + list(range(1380, 2886)):
    tc = instance.copy()
    tc[3]= val
    vv = lf.model.predict([tc, instance])
    if 1 in (vv[0]-vv[1]) or -1 in (vv[0]-vv[1]):
        print(val, vv[0],vv[1])

In [55]:
# Torque [Nm]:
for val in list(range(484, 652, 1))+list(range(655, 766, 1)):
    tc = instance.copy()
    tc[4]= val/10
    vv = lf.model.predict([tc, instance])
    if 1 in (vv[0]-vv[1]) or -1 in (vv[0]-vv[1]):
        print(val, vv[0],vv[1])

484 [1 0 0 0] [1 0 1 1]
485 [0 0 0 1] [1 0 1 1]
486 [0 0 0 1] [1 0 1 1]
487 [0 0 0 1] [1 0 1 1]
488 [0 0 0 1] [1 0 1 1]
489 [0 0 0 1] [1 0 1 1]
490 [0 0 0 1] [1 0 1 1]
491 [0 0 0 1] [1 0 1 1]
492 [0 0 0 1] [1 0 1 1]
493 [0 0 0 1] [1 0 1 1]
494 [0 0 0 1] [1 0 1 1]
495 [0 0 0 1] [1 0 1 1]
496 [0 0 0 1] [1 0 1 1]
497 [0 0 0 1] [1 0 1 1]
498 [0 0 0 1] [1 0 1 1]
499 [0 0 0 1] [1 0 1 1]
500 [0 0 0 1] [1 0 1 1]
501 [0 0 0 1] [1 0 1 1]
502 [0 0 0 1] [1 0 1 1]
503 [0 0 0 1] [1 0 1 1]
504 [0 0 0 1] [1 0 1 1]
505 [0 0 0 1] [1 0 1 1]
506 [0 0 0 1] [1 0 1 1]
507 [0 0 0 1] [1 0 1 1]
508 [0 0 0 1] [1 0 1 1]
509 [0 0 0 1] [1 0 1 1]
510 [0 0 0 1] [1 0 1 1]
511 [0 0 0 1] [1 0 1 1]
512 [0 0 0 1] [1 0 1 1]
513 [0 0 0 1] [1 0 1 1]
514 [0 0 0 1] [1 0 1 1]
515 [0 0 0 1] [1 0 1 1]
516 [0 0 0 1] [1 0 1 1]
517 [0 0 0 1] [1 0 1 1]
518 [0 0 0 1] [1 0 1 1]
519 [0 0 0 1] [1 0 1 1]
520 [0 0 0 1] [1 0 1 1]
521 [0 0 0 1] [1 0 1 1]
522 [0 0 0 1] [1 0 1 1]
523 [0 0 0 1] [1 0 1 1]
524 [0 0 0 1] [1 0 1 1]
525 [0 0 0 1] [1

In [53]:
# Tool wear [min]:
for val in list(range(1765, 2075, 1))+list(range(2084, 2531, 1)):
    tc = instance.copy()
    tc[5]= val/10
    vv = lf.model.predict([tc, instance])
    if 1 in (vv[0]-vv[1]) or -1 in (vv[0]-vv[1]):
        print(val/10, vv[0],vv[1])

176.5 [0 0 1 0] [1 0 1 1]
